## Evaluación 

En este proyecto consideramos tres dimensiones clave para seleccionar el modelo más adecuado: tiempo de procesamiento, costos y precisión.

**Tiempo de procesamiento**
El tiempo que cada modelo tardó en procesar la carpeta de Cartas Documento fue muy similar entre las tres alternativas. Esta homogeneidad hace que, para este caso, el tiempo no sea un criterio relevante de decisión.

**Costos**
Tesseract OCR y Doctr se ejecutaron en la computadora local sin requerir recursos adicionales ni gastos en infraestructura o licencias.

API de GPT implicó un costo variable asociado al consumo. Cada iteración completa sobre la carpeta de Cartas Documento (conversión imagen → texto) tuvo un costo aproximado de 0,66 USD.

**Precisión**
Esta es la métrica más compleja de estimar, ya que requiere un dataset de referencia (ground truth) para comparar las transcripciones.

Como paso previo, generaremos un dataset target con transcripciones verificadas manualmente.

Una vez disponible, calcularemos la distancia de Levenshtein entre cada transcripción generada por los modelos y el texto objetivo, obteniendo así una medida cuantitativa y comparable de la precisión de cada enfoque.






#### Construcción del target
 Antes de abordar el problema de la evaluación de los outputs, vamos a extraer entidades de los achivos transcriptos a mano. De esa manera, construimos nuestro target para poder comparar. Utilizamos la misma lógica de extracción de entidades que en los otros casos y usamos el modelo Llama 3. 

In [1]:
from pydantic import BaseModel
from typing import Optional
from pydantic import ValidationError
import re, unicodedata
from tqdm import tqdm
from requests.exceptions import ReadTimeout
from pathlib import Path
import json
import csv
import requests
from requests.exceptions import ReadTimeout

In [15]:
TXT_FOLDER = Path("Transcripciones_manual")
OUTPUT_CSV = Path("entidades_extraidas_mauscritas.csv")
PROMPT_FILE = "prompt_2.txt"
MODEL       = "llama3:latest"
OLLAMA_URL  = "http://localhost:11434/api/generate"
TEMPERATURE = 0.0
TIMEOUT     = 900
MAX_CHARS   = 9000

In [17]:
class Entity(BaseModel):
    Remitente: Optional[str] = None
    DNI: Optional[str] = None
    CUIT_CUIL: Optional[str] = None
    Cuerpo: str


In [18]:
CLEAN_RE = re.compile(r"[^\w\s.,;:()@€$%/-]")

def clean_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = CLEAN_RE.sub("", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

In [19]:
JSON_SCHEMA = {
    "type": "object",
    "properties": {
        "Remitente": {"type": ["string", "null"]},
        "DNI": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "CUIT_CUIL": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "Cuerpo": {"type": "string"},
    },
    "required": ["Cuerpo"],
    "additionalProperties": False,
}

with open(PROMPT_FILE, encoding="utf-8") as f:
    PROMPT = f.read()

def call_ollama(text: str) -> dict:
    payload = {
        "model": MODEL,
        "prompt": f"<|system|>\n{PROMPT}<|end|>\n<|user|>\n{text[:MAX_CHARS]}<|end|>\n<|assistant|>",
        "format": "json",
        "stream": False,
        "options": {"temperature": TEMPERATURE, "json_schema": JSON_SCHEMA},
    }
    try:
        r = requests.post(OLLAMA_URL, json=payload, timeout=TIMEOUT)
    except ReadTimeout:
        raise TimeoutError("Timeout de Ollama")
    r.raise_for_status()
    return json.loads(r.json()["response"])

def main():
    dir_path = Path(TXT_FOLDER)
    if not dir_path.is_dir():
        raise SystemExit(f"Ruta no encontrada: {dir_path}")

    rows = []
    for file in tqdm(sorted(dir_path.glob("*.txt")), desc="TXT"):
        raw = file.read_text(encoding="utf-8", errors="ignore")
        cleaned = clean_text(raw)
        try:
            data = call_ollama(cleaned)
            ent = Entity.model_validate(data)
            rows.append({
                "ARCHIVO": file.name,
                "Remitente": (ent.Remitente or "NaN").strip() or "NaN",
                "DNI": re.sub(r"\D", "", ent.DNI or "") or "NaN",
                "CUIT_CUIL": re.sub(r"\D", "", ent.CUIT_CUIL or "") or "NaN",
                "Cuerpo": ent.Cuerpo.strip() or "NaN",
            })
        except Exception as e:
            print(f"❌ {file.name}: {e}")
    if rows:
        with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
            csv.DictWriter(f, fieldnames=rows[0].keys()).writeheader(); csv.DictWriter(f, fieldnames=rows[0].keys()).writerows(rows)
        print(f"✅ CSV generado en {OUTPUT_CSV} ({len(rows)} filas)")
    else:
        print("No se extrajeron entidades válidas.")

In [ ]:
if __name__ == "__main__":
    main()

TXT:   0%|          | 0/7 [00:00<?, ?it/s]

### Distancia de Levenshtein

Una vez realizada la extracción de entidades con cada método, evaluaremos cuál obtuvo mejores resultados utilizando la **distancia de Levenshtein**.  

Esta métrica mide el número mínimo de operaciones necesarias para transformar una cadena en otra. Las operaciones permitidas son:  
- **Inserción** de un carácter.  
- **Eliminación** de un carácter.  
- **Sustitución** de un carácter por otro.  

Un valor **0** indica coincidencia exacta. Cuanto mayor sea el valor, mayor es la diferencia entre las cadenas.  




1. Instalamos librerías necesarias

In [ ]:
#pip install pandas

In [ ]:
#pip install jellyfish

In [16]:
from __future__ import annotations
import sys
import re
import unicodedata
from pathlib import Path
from typing import Optional, Dict
import pandas as pd

try:
    import jellyfish
except ModuleNotFoundError:
    raise SystemExit(
        f"[ERROR] jellyfish no está instalado en este intérprete:\n  {sys.executable}\n"
        f"Instalá con:\n  \"{sys.executable}\" -m pip install jellyfish"
    )

2. Definimos el path

In [39]:
BASE_DIR = Path.cwd()

CSV_TARGET_MANUS = BASE_DIR / "entidades_extraidas_manuscritas.csv"
CSV_BASES: Dict[str, Path] = {
    "DOCTR":     BASE_DIR / "entidades_extraidas_DOCTR.csv",
    "GPT-5":     BASE_DIR / "entidades_extraidas_GPT-5.csv",
    "Tesseract": BASE_DIR / "entidades_extraidas_Tesseract.csv"
}

3. Definimos funciones de normalización para cada variable, para hacerlas más comparables entre sí.

`quitar_acentos(text: str) -> str`
Elimina acentos de las letras y preservamos la letra Ñ


`MARCADORES_CUERPO`
Lista de patrones regex que representan marcadores específicos a eliminar del cuerpo del texto. Esta función elimina cualquier caracter que no sea: letras, números, espacios, comas y puntos.

In [40]:
def quitar_acentos(text: str) -> str:
    if text is None:
        return ""
    text = str(text).replace("ñ", "__enie__").replace("Ñ", "__ENIE__")
    sin_acento = "".join(
        c for c in unicodedata.normalize("NFD", text)
        if unicodedata.category(c) != "Mn"
    )
    return sin_acento.replace("__enie__", "ñ").replace("__ENIE__", "Ñ")


MARCADORES_CUERPO = [r"\(\?\)"]  

def norm_cuerpo(x: str) -> str:
    if not x:
        return ""
    t = quitar_acentos(x).lower()
    for pat in MARCADORES_CUERPO:
        t = re.sub(rf"\s*{pat}\s*", " ", t)
    # limpieza general: letras (incluyendo ñ), números, espacios, comas y puntos
    t = re.sub(r"[^a-z0-9ñÑ\s.,]", "", t)
    # colapsar espacios
    return " ".join(t.split())


4. Funciones de lectura: Estas dos funciones homogeneizan los CSV de entrada y construyen una clave de emparejamiento por nombre de archivo. Esto permite que todos los datasets compartan el mismo esquema de columnas y una clave (archivo_norm) comparable entre sí.

`estandarizar_cols` limpia y pone en formato estándar el DataFrame (nombres de columnas, tipos, clave archivo_norm).

`cargar_csv` asegura que el archivo exista, lo lee como texto, lo estandariza y valida que tenga todo lo necesario para las comparaciones.

In [41]:
def estandarizar_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={c: (c.strip() if isinstance(c, str) else c) for c in df.columns})
    lower = {str(c).lower(): c for c in df.columns}

    if 'archivo' in lower:
        df = df.rename(columns={lower['archivo']: 'archivo'})
    elif 'ARCHIVO' in df.columns:
        df = df.rename(columns={'ARCHIVO': 'archivo'})

    if 'remitente' in lower and 'Remitente' not in df.columns:
        df = df.rename(columns={lower['remitente']: 'Remitente'})
    if 'cuerpo' in lower and 'Cuerpo' not in df.columns:
        df = df.rename(columns={lower['cuerpo']: 'Cuerpo'})
    if 'dni' in lower and 'DNI' not in df.columns:
        df = df.rename(columns={lower['dni']: 'DNI'})
    for k in ['cuit_cuil','cuitcuil','cuit','cuil']:
        if k in lower and 'CUIT_CUIL' not in df.columns:
            df = df.rename(columns={lower[k]: 'CUIT_CUIL'})
            break
    if 'CUIT_CUIL' not in df.columns and 'Cuit_Cuil' in df.columns:
        df = df.rename(columns={'Cuit_Cuil': 'CUIT_CUIL'})

    # tipado y limpieza básica
    for c in ['archivo','Remitente','DNI','CUIT_CUIL','Cuerpo']:
        if c in df.columns:
            df[c] = df[c].astype('string')

    # filtrar filas sin archivo y generar clave normalizada
    if 'archivo' in df.columns:
        df = df[df['archivo'].notna() & (df['archivo'].str.strip()!='')].copy()
        df['archivo_norm'] = df['archivo'].map(norm_fname)

    return df


def cargar_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"No existe el archivo: {path}")
    df = pd.read_csv(path, encoding='utf-8-sig', dtype=str)
    df = estandarizar_cols(df)
    req = {'archivo','Remitente','DNI','CUIT_CUIL','Cuerpo','archivo_norm'}
    faltan = req - set(df.columns)
    if faltan:
        raise ValueError(f"Faltan columnas en {path.name}: {faltan}. Encontradas: {list(df.columns)}")
    return df

5.  Este bloque hace la comparación de las 7 cartas manuscritas contra cada base OCR (DOCTR, GPT‑5, Tesseract), calculando distancias de Levenshtein por campo. 

`comparar_base_con_target` Compara el target manuscrito (lado izquierdo) contra la base OCR (lado derecho), manteniendo SIEMPRE todas las filas del target. Luego calcula las distancias de similitud campo a campo.

`comparar_bases_vs_target` Orquesta la comparación del target manuscrito contra todas las bases OCR disponibles. Para ello, aplica la normalización (norm_nombre, norm_cuerpo, norm_num) en los campos clave, llama a comparar_base_con_target para obtener distancias y consolida los resultados.

In [42]:
def comparar_base_con_target(df_target: pd.DataFrame, df_base: pd.DataFrame, nombre_base: str) -> pd.DataFrame:
        # Renombrar columnas para claridad
    t = df_target[['archivo','Remitente','DNI','CUIT_CUIL','Cuerpo','archivo_norm']].copy().rename(
        columns={'archivo':'archivo_target','Remitente':'Remitente_target','DNI':'DNI_target','CUIT_CUIL':'CUIT_CUIL_target','Cuerpo':'Cuerpo_target'}
    )
    b = df_base[['archivo','Remitente','DNI','CUIT_CUIL','Cuerpo','archivo_norm']].copy().rename(
        columns={'archivo':'archivo_base','Remitente':'Remitente_base','DNI':'DNI_base','CUIT_CUIL':'CUIT_CUIL_base','Cuerpo':'Cuerpo_base'}
    )

    # Merge con left join desde el target 
    m = t.merge(b[['archivo_norm','archivo_base','Remitente_base','DNI_base','CUIT_CUIL_base','Cuerpo_base']],
                on='archivo_norm', how='left')

    dist_rem, dist_cue, dist_cue_norm, dist_dni, dist_cuit = [], [], [], [], []
    for _, r in m.iterrows():
        # Remitente
        rt = norm_nombre(r.get('Remitente_target'))
        rb = norm_nombre(r.get('Remitente_base')) if pd.notna(r.get('Remitente_base')) else None
        drem = lev(rt, rb); dist_rem.append(pd.NA if drem is None else drem)

        # Cuerpo
        ct = norm_cuerpo(r.get('Cuerpo_target'))
        cb = norm_cuerpo(r.get('Cuerpo_base')) if pd.notna(r.get('Cuerpo_base')) else None
        dcue = lev(ct, cb); dist_cue.append(pd.NA if dcue is None else dcue)
        if cb is None:
            dist_cue_norm.append(pd.NA)
        else:
            mlen = max(len(ct or ""), len(cb or "")); dist_cue_norm.append(pd.NA if mlen==0 else dcue/mlen)

        # DNI
        dt = norm_num(r.get('DNI_target'))
        db = norm_num(r.get('DNI_base')) if pd.notna(r.get('DNI_base')) else None
        ddni = lev(dt, db); dist_dni.append(pd.NA if ddni is None else ddni)

        # CUIT
        qt = norm_num(r.get('CUIT_CUIL_target'))
        qb = norm_num(r.get('CUIT_CUIL_base')) if pd.notna(r.get('CUIT_CUIL_base')) else None
        dcui = lev(qt, qb); dist_cuit.append(pd.NA if dcui is None else dcui)

    out = pd.DataFrame({
        'ARCHIVO_TARGET': m['archivo_target'],
        'ARCHIVO_BASE': m['archivo_base'],
        'archivo_norm': m['archivo_norm'],
        'base': nombre_base,
        'dist_remitente': pd.Series(dist_rem, dtype='Int64'),
        'dist_cuerpo': pd.Series(dist_cue, dtype='Int64'),
        'dist_cuerpo_norm': pd.Series(dist_cue_norm, dtype='Float64'),
        'dist_dni': pd.Series(dist_dni, dtype='Int64'),
        'dist_cuit': pd.Series(dist_cuit, dtype='Int64'),
    })
    return out


def comparar_bases_vs_target(csv_target: Path = CSV_TARGET_MANUS, csv_bases: Dict[str, Path] = CSV_BASES) -> pd.DataFrame:
    df_target = cargar_csv(csv_target)

    df_target['Remitente'] = df_target['Remitente'].map(norm_nombre)
    df_target['Cuerpo'] = df_target['Cuerpo'].map(norm_cuerpo)
    df_target['DNI'] = df_target['DNI'].map(norm_num)
    df_target['CUIT_CUIL'] = df_target['CUIT_CUIL'].map(norm_num)

    resultados = []
    for nombre_base, ruta in csv_bases.items():
        df_base = cargar_csv(ruta)
        df_base['Remitente'] = df_base['Remitente'].map(norm_nombre)
        df_base['Cuerpo'] = df_base['Cuerpo'].map(norm_cuerpo)
        df_base['DNI'] = df_base['DNI'].map(norm_num)
        df_base['CUIT_CUIL'] = df_base['CUIT_CUIL'].map(norm_num)

        df_cmp = comparar_base_con_target(df_target, df_base, nombre_base)
        if not df_cmp.empty:
            resultados.append(df_cmp)

    cols = ['ARCHIVO_TARGET','ARCHIVO_BASE','archivo_norm','base',
            'dist_remitente','dist_cuerpo','dist_cuerpo_norm','dist_dni','dist_cuit']
    if not resultados:
        return pd.DataFrame(columns=cols)

    out = pd.concat(resultados, ignore_index=True)[cols]
    out = out.sort_values(['ARCHIVO_TARGET','base'], kind='mergesort')
    return out

Ejecutamos

In [43]:
if __name__ == "__main__":
    print("BASE_DIR:", BASE_DIR)
    print("Target:", CSV_TARGET_MANUS, "->", "OK" if CSV_TARGET_MANUS.exists() else "NO")
    for k, v in CSV_BASES.items():
        print(f"Base {k}:", v, "->", "OK" if v.exists() else "NO")

    df_out = comparar_bases_vs_target(CSV_TARGET_MANUS, CSV_BASES)
    print(f"\n[OK] Comparadas {df_out['base'].nunique()} bases contra target manuscritas, sobre {df_out['ARCHIVO_TARGET'].nunique()} archivos.")
    print(df_out.head(20).to_string(index=False))

BASE_DIR: d:\Formación\Managment & Analytics - ITBA\15. Deep Learning\Trabajo_Final
Target: d:\Formación\Managment & Analytics - ITBA\15. Deep Learning\Trabajo_Final\entidades_extraidas_manuscritas.csv -> OK
Base DOCTR: d:\Formación\Managment & Analytics - ITBA\15. Deep Learning\Trabajo_Final\entidades_extraidas_DOCTR.csv -> OK
Base GPT-5: d:\Formación\Managment & Analytics - ITBA\15. Deep Learning\Trabajo_Final\entidades_extraidas_GPT-5.csv -> OK
Base Tesseract: d:\Formación\Managment & Analytics - ITBA\15. Deep Learning\Trabajo_Final\entidades_extraidas_Tesseract.csv -> OK

[OK] Comparadas 3 bases contra target manuscritas, sobre 7 archivos.
         ARCHIVO_TARGET            ARCHIVO_BASE       archivo_norm      base  dist_remitente  dist_cuerpo  dist_cuerpo_norm  dist_dni  dist_cuit
      Bruno23689270.txt       Bruno23689270.txt      bruno23689270     DOCTR               3          897          0.306248      <NA>       <NA>
      Bruno23689270.txt       Bruno23689270.txt      bruno

Vamos a calcular un precisión global por modelo. 
Tomamos un umbral de dist_cuerpo_norm <= 0.10
Es decir, hasta un 10 % de diferencia respecto al manuscrito

In [46]:
def precision_global_cuerpo(df_out: pd.DataFrame, umbral: float = 0.10) -> pd.DataFrame:
    df = df_out.copy()
    df['acierto_cuerpo'] = df['dist_cuerpo_norm'].le(umbral)
    
    return (
        df.groupby('base', dropna=False)
          .agg(
              total=('acierto_cuerpo', 'size'),
              aciertos=('acierto_cuerpo', 'sum')
          )
          .assign(
              precision_global=lambda d: (d['aciertos'] / d['total'] * 100).round(1)
          )
          .reset_index()
    )

# Ejemplo de uso:
precision_cuerpos = precision_global_cuerpo(df_out)
print(precision_cuerpos)


        base  total  aciertos  precision_global
0      DOCTR      7         4              57.1
1      GPT-5      7         2              28.6
2  Tesseract      7         4              57.1


## Resultados
La comparación de las 7 cartas manuscritas contra las salidas de DOCTR, GPT‑5 y Tesseract muestra comportamientos heterogéneos en la extracción de texto y datos.

1. **Cobertura**: salvo un caso (EVERTEC30707869484), todas las cartas manuscritas tienen correspondencia en las tres bases OCR. La ausencia de match en DOCTR para esta carta fue porque el modelo se exedió del timeout.

2. **Remitente**: GPT-5 obtiene distancia cero (mejor coincidencia posible) en 3 cartas (Bruno400212294002.txt, Bruno400212294002_1.txt, Pantano28462989.txt). No obstante, esto ocurre porque en el archivo de target y en GPT estos valores están en NaN (vacíos).

3. **Distancias en cuerpo**: Tesseract es el más consistente, ya que obtiene la menor distancia normalizada en 4 de las 6 cartas con datos (Bruno23689270.txt, Bruno400212294002.txt, Bruno400212294002_1.txt, Coyle401547680601.txt). DOCTR tiene mejor desempeño en Pantano28462989.txt (0.0086 vs 0.0665 de GPT-5). GPT-5 sólo logra liderar en una carta (Mallo12980371.txt frente a DOCTR y Tesseract), pero en general su distancia normalizada es más alta, lo que indica menor coincidencias.

4. **DNI y CUIT**: gran parte de las comparaciones no poseen valores en estos campos, lo que limita la evaluación de exactitud. 

## **Conclusión** 
DOCTR y Tesseract logran, en general, mejor alineación con las cartas manuscritas en remitente y cuerpo, con distancias más bajas y consistentes. GPT‑5, si bien acierta en ciertos campos, presenta variabilidad alta y errores significativos en el cuerpo en varios documentos. Este resultado sugiere que, para manuscritos de este tipo, los OCR clásicos bien configurados parecieran ofrecer salidas más consistentes que la transcripción vía modelo de lenguaje, especialmente en documentos con formato y legibilidad más comprometida.

